In [ ]:
from __init__ import *

In [ ]:
ws, hs = read_ws_hs("../output/mf", SETTING["model_mode"], SETTING["model_data_dicts"],)

In [ ]:
distance_function = kraft.information.get_icd

size = 800

layout = {"height": size, "width": size}

In [ ]:
w_node_x_node_d_s = []

for i, w in enumerate(ws):

    labels = w.columns

    m = w.to_numpy()

    d = kraft.array.apply_function_on_slices_from_2_matrices(
        m, m, 1, distance_function,
    )

    w_node_x_node_d_s.append(d)

    kraft.plot.plot_heat_map(
        pd.DataFrame(data=d, index=labels, columns=labels),
        layout={"title": {"text": "W {} Distance".format(i)}, **layout},
    )

In [ ]:
h_node_x_node_d_s = []

for i, h in enumerate(hs):

    labels = h.index

    m = h.to_numpy()

    d = kraft.array.apply_function_on_slices_from_2_matrices(
        m, m, 0, distance_function,
    )

    h_node_x_node_d_s.append(d)

    kraft.plot.plot_heat_map(
        pd.DataFrame(data=d, index=labels, columns=labels),
        layout={"title": {"text": "H {} Distance".format(i)}, **layout},
    )

In [ ]:
node_x_node_d_s = np.concatenate((w_node_x_node_d_s, h_node_x_node_d_s))

node_x_node_d = pd.DataFrame(
    data=np.mean(node_x_node_d_s, axis=0), index=hs[0].index, columns=ws[0].columns
)

kraft.plot.plot_heat_map(
    node_x_node_d, layout={"title": {"text": "W H Distance"}, **layout},
)

In [ ]:
rs_01 = kraft.array.normalize(
    np.concatenate(tuple((r,) * r for r in SETTING["mf_rs"])), "0-1"
)

node_factors = SETTING["mf_r_range"] ** rs_01

node_marker_size = 3 ** (1 - rs_01) * 16

In [ ]:
directory_path = "../output/gps_map/"

kraft.path.path(directory_path)

for w_h_str in ("w", "h"):

    if w_h_str == "w":

        element_x_node = pd.concat(ws)

    else:

        element_x_node = pd.concat(hs, axis=1).T

    element_x_node *= node_factors

    element_x_node **= SETTING["{}_gps_map_node_power".format(w_h_str)]

    gps_map = kraft.gps_map.GPSMap(node_x_node_d, element_x_node,)

    kraft.gps_map.write_gps_map(
        "{}/{}_gps_map.pickle.gz".format(directory_path, w_h_str), gps_map
    )

    gps_map.plot(
        point_trace=SETTING["{}_gps_map_element_trace".format(w_h_str)],
        html_file_path="{}/{}_gps_map.html".format(directory_path, w_h_str),
    )